In [1]:

from copy import deepcopy

import torch.optim as optim
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from tqdm import tqdm
import time

In [2]:
from senmodel.model.utils import *
from senmodel.metrics.nonlinearity_metrics import *
from senmodel.metrics.edge_finder import *
from senmodel.metrics.train_metrics import *
from senmodel.train.train import *

In [3]:
torch.manual_seed(0)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [4]:
class SimpleFCN(nn.Module):
    def __init__(self, input_size=28 * 28, hidden_size=16):
        super(SimpleFCN, self).__init__()
        self.fc0 = nn.Linear(input_size, 10)
        # self.fc1 = nn.Linear(hidden_size, 10)
        self.act = nn.ReLU()

    def forward(self, x):
        x = self.fc0(x)
        return x

In [5]:
# Dataset and Dataloader
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.view(-1))
])

# Load dataset and split into train/validation sets
dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)

train_dataset, val_dataset, test_dataset = random_split(dataset, [0.6, 0.2, 0.2])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [6]:
next(iter(train_loader))[0].shape

torch.Size([64, 784])

In [7]:
model = SimpleFCN()
sparse_model = convert_dense_to_sparse_network(model, layers=[model.fc0], device=device)

In [8]:
hyperparams = {
    "num_epochs": 64,
    "metric": AbsGradientEdgeMetric(nn.CrossEntropyLoss()),
    "aggregation_mode": "mean",
    "choose_thresholds": {"fc0": 0.7}, # 1.0 -> all edges, 0.0 -> no edges
    "threshold": 0.005,
    "min_delta_epoch_replace": 8,
    "window_size": 5,
    "lr": 1e-4,
    "delete_after": 2,    
    "task_type": "classification",
    "fully_connected": False,
    "max_to_replace": 900 # None -> no limit
}

name = ", ".join(
    f"{key}: {value.__class__.__name__ if key == 'metric' else value}"
    for key, value in hyperparams.items()
)

name

"num_epochs: 64, metric: AbsGradientEdgeMetric, aggregation_mode: mean, choose_thresholds: {'fc0': 0.7}, threshold: 0.005, min_delta_epoch_replace: 8, window_size: 5, lr: 0.0001, delete_after: 2, task_type: classification, fully_connected: False, max_to_replace: 900"

In [9]:
import wandb

wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: fedornigretuk to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [10]:
wandb.finish()
run = wandb.init(
    project="self-expanding-nets",
    name=f"trash",
    config=hyperparams
)

In [ ]:
criterion = nn.CrossEntropyLoss()
train_sparse_recursive(sparse_model, train_loader, train_loader, val_loader, criterion, hyperparams, device)

100%|██████████| 563/563 [00:06<00:00, 83.96it/s]


Epoch 1/64, Train Loss: 1.5927, Val Loss: 1.1426, Val Accuracy: 0.8047


100%|██████████| 563/563 [00:06<00:00, 85.38it/s]


Epoch 2/64, Train Loss: 0.9270, Val Loss: 0.7927, Val Accuracy: 0.8404


100%|██████████| 563/563 [00:06<00:00, 85.44it/s]


...

Epoch 37/64, Train Loss: 0.1708, Val Loss: 0.1945, Val Accuracy: 0.9453


100%|██████████| 563/563 [00:15<00:00, 37.01it/s]


Epoch 38/64, Train Loss: 0.1657, Val Loss: 0.1913, Val Accuracy: 0.9447
Chosen edges: tensor([[   3,    8,    9,    2,    3,    8,    9,    2,    3,    7,    8,    9,
            3,    8,    9,    2,    3,    2,    3,    8,    9,    5,    3,    3,
            8,    2,    3,    8,    8,    3,    8,    9,    2,    3,    7,    8,
            9,    8,    2,    3,    8,    9,    2,    3,    4,    5,    7,    8,
            9,    8,    8,    2,    7,    8,    9],
        [ 789,  789,  789,  791,  791,  791,  791,  832,  832,  832,  832,  832,
          836,  836,  836,  857,  857,  865,  865,  865,  865,  907,  927,  929,
          929,  930,  930,  930,  931,  939,  939,  939,  983,  983,  983,  983,
          983,  989,  990,  990,  990,  990,  991,  991,  991,  991,  991,  991,
          991, 1029, 1030, 1102, 1102, 1102, 1102]]) 55


100%|██████████| 563/563 [00:17<00:00, 31.34it/s]


Epoch 39/64, Train Loss: 0.1612, Val Loss: 0.1857, Val Accuracy: 0.9473
